Example usage of the provided "cython_peg.py" parsing file.  This code is not mature/complex enough to be a python module.  
The parsing will probably need to be adjusted for specific project needs.  The pyparsing syntax is dense but powerful.  
It will take some document reading to learn the pyparsing syntax and make edits.

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from cython_peg import cython2stub
from pathlib import Path

# basic import template
# working on dynamic imports
import_template = "\n".join([
"import numpy as np",
"from enum import Enum",
"from typing import Optional, Tuple, Dict, List, Generator, Union, DefaultDict",
"\n# type convertions from cython to python",
"float64 = float32 = double = long = longlong = float",
"uint64 = uint32 = uint16 = uint8 = short = int",
"COLON = str",
"\n"])

examples = Path(r"../examples/").glob("*.pyx")
for example in examples:
    
    # generate stub file
    stub_file = cython2stub(example)
    
    # save the type file
    with open(example.with_suffix(".pyi"), mode='w') as f:
        f.write(import_template+stub_file)
        
    # print output
    print(stub_file)


def add_and_multiply(x: double, y: double) -> double:
    ...

def fibonacci(n: int) -> int:
    """some sort of algorithm?"""
    ...

def sum_array(arr: int[COLON]) -> int:
    ...

class State(Enum):
    state1=1
    state2=2

class Point:

    class BabyPoint:

        class BabyPoint2:

            def distance_to_baby(self, y: double) -> double:
                ...


        def distance_to_baby(self, y: double) -> double:
            ...


    def distance_to_origin(self) -> double:
        """calculate some distance stuff"""
        ...

    def distance_to_point(self, x2: double, y2: double) -> double:
        ...



def compute_sin(angle: double) -> double:
    ...


